# Compute N<sub>2</sub> fluxes from ERA5 heat fluxes

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import yaml

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

import intake
import cdsapi

import regrid_tools
import config
import solubility
import util

ModuleNotFoundError: No module named 'cdsapi'

In [ ]:
year0, yearf = 1984, 2020
scaleby = 1 / 1.3
shift_time = 0.

## Get the heat flux

Get data from ECMWF servers using `cdsapi`. Generate a command here: https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-monthly-means?tab=download

Interpolate onto a more sensible grid. The ERA5 data is already interpolated from the native Gaussian grid using bilinear interpolation. They serve the data with values at -90°N and 90°N: the data in these top and bottom rows is all identical.

Use `sst` field to generate land mask. The following test passes:
```python
for i in range(len(ds.time)):
    assert (ds.tos.isel(time=0).isnull() == ds.tos.isel(time=i).isnull()).all()
```

Variable definitions (see [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-monthly-means?tab=overview)):
```
mslhf: Mean surface latent heat flux
msnlwrf: Mean surface net long-wave radiation flux
msnswrf: Mean surface net short-wave radiation flux
msshf: Mean surface sensible heat flux
sst: Sea surface temperature
```

| Variable         | Short Name | Units     | Description |
| ---------------- | ---------- | --------- | ----------- |
|Mean surface latent heat flux | `mslhf` | W m<sup>-2</sup> | This parameter is the transfer of latent heat (resulting from water phase changes, such as evaporation or condensation) between the Earth's surface and the atmosphere through the effects of turbulent air motion. Evaporation from the Earth's surface represents a transfer of energy from the surface to the atmosphere. This parameter is a mean over a particular time period (the processing period) which depends on the data extracted. For the monthly averaged reanalysis and the monthly averaged ensemble members, the processing period is the complete, whole month. For the monthly averaged reanalysis by hour of day, the processing period is 1 hour for every day of the month and for the monthly averaged ensemble members by hour of day, the processing period is 3 hours for every day of the month. The ECMWF convention for vertical fluxes is positive downwards.|
| Mean surface net long-wave radiation flux| `msnlwrf` | W m<sup>-2</sup> |Thermal radiation (also known as longwave or terrestrial radiation) refers to radiation emitted by the atmosphere, clouds and the surface of the Earth. This parameter is the difference between downward and upward thermal radiation at the surface of the Earth. It is the amount of radiation passing through a horizontal plane. The atmosphere and clouds emit thermal radiation in all directions, some of which reaches the surface as downward thermal radiation. The upward thermal radiation at the surface consists of thermal radiation emitted by the surface plus the fraction of downwards thermal radiation reflected upward by the surface. This parameter is a mean over a particular time period (the processing period) which depends on the data extracted. For the monthly averaged reanalysis and the monthly averaged ensemble members, the processing period is the complete, whole month. For the monthly averaged reanalysis by hour of day, the processing period is 1 hour for every day of the month and for the monthly averaged ensemble members by hour of day, the processing period is 3 hours for every day of the month. The ECMWF convention for vertical fluxes is positive downwards.|
| Mean surface net short-wave radiation flux | `msnswrf` | W m<sup>-2</sup> | This parameter is the amount of solar radiation (also known as shortwave radiation) that reaches a horizontal plane at the surface of the Earth (both direct and diffuse) minus the amount reflected by the Earth's surface (which is governed by the albedo). Radiation from the Sun (solar, or shortwave, radiation) is partly reflected back to space by clouds and particles in the atmosphere (aerosols) and some of it is absorbed. The remainder is incident on the Earth's surface, where some of it is reflected. This parameter is a mean over a particular time period (the processing period) which depends on the data extracted. For the monthly averaged reanalysis and the monthly averaged ensemble members, the processing period is the complete, whole month. For the monthly averaged reanalysis by hour of day, the processing period is 1 hour for every day of the month and for the monthly averaged ensemble members by hour of day, the processing period is 3 hours for every day of the month. The ECMWF convention for vertical fluxes is positive downwards.|
| Mean surface sensible heat flux | `msshf` | W m<sup>-2</sup>  | 	This parameter is the transfer of heat between the Earth's surface and the atmosphere through the effects of turbulent air motion (but excluding any heat transfer resulting from condensation or evaporation). The magnitude of the sensible heat flux is governed by the difference in temperature between the surface and the overlying atmosphere, wind speed and the surface roughness. For example, cold air overlying a warm surface would produce a sensible heat flux from the land (or ocean) into the atmosphere. This parameter is a mean over a particular time period (the processing period) which depends on the data extracted. For the monthly averaged reanalysis and the monthly averaged ensemble members, the processing period is the complete, whole month. For the monthly averaged reanalysis by hour of day, the processing period is 1 hour for every day of the month and for the monthly averaged ensemble members by hour of day, the processing period is 3 hours for every day of the month. The ECMWF convention for vertical fluxes is positive downwards.|

In [ ]:
%%time
import urllib3
urllib3.disable_warnings()

file_out = f"{config.intake_cachedir}/era5.mean_surface_heat_fluxes.{year0:04d}-{yearf:04d}.nc"

if not os.path.exists(file_out):
    
    try:
        with open("ecmwf_credentials.yaml") as fid:
            credentials = yaml.safe_load(fid)
    except:
        raise    
    
    c = cdsapi.Client(**credentials)
    c.retrieve(
        'reanalysis-era5-single-levels-monthly-means',
        {
            'format': 'netcdf',
            'variable': [
                'mean_surface_latent_heat_flux', 
                'mean_surface_net_long_wave_radiation_flux', 
                'mean_surface_net_short_wave_radiation_flux',
                'mean_surface_sensible_heat_flux',
                'sea_surface_temperature',
            ],
            'year': [f"{y:04d}" for y in range(year0, yearf+1)],
            'month': [f"{m:02d}" for m in range(1, 13)],
            'time': '00:00',
            'product_type': 'monthly_averaged_reanalysis',
        },
        file_out)

ds = xr.open_dataset(file_out)
grid = util.generate_latlon_grid(1440, 720)

# turn the world right-side up
ds = ds.reindex(latitude=ds.latitude[::-1])

# mask out land
ds = ds.where(ds.sst)

# compute total net heat flux
ds["hfds"] = ds.mslhf + ds.msnlwrf + ds.msnswrf + ds.msshf
ds.hfds.attrs = ds.mslhf.attrs
ds.hfds.attrs["long_name"] = "Net surface heat flux"

ds = ds.rename({"sst": "tos"})
ds = ds[["hfds", "tos"]].interp(longitude=grid.lon, latitude=grid.lat)
ds["area"] = grid.area.where(ds.tos.isel(time=0).notnull()).fillna(0.)

with xr.set_options(keep_attrs=True):
    ds['tos'] = ds.tos - solubility.T0_Kelvin
    ds.tos.attrs['units'] = 'degC'

ds

### Examine this dataset

In [ ]:
ds.area.plot()

## Get WOA salinity data

In [ ]:
ds.tos.isel(time=0).plot()

In [ ]:
ds.hfds.isel(time=0).plot()

In [ ]:
ds.hfds.mean('time').plot()

In [ ]:
cat = intake.open_catalog("catalogs/woa2018-catalog.yml")
print(cat.yaml())

In [ ]:
%%time
def retrieve_dataset(value):
    try:
        ds = cat.salinity(time_code=value).to_dask().isel(depth=0, drop=True)
    except Exception:
        ds = None
        raise
    return ds


ds_list = list(map(retrieve_dataset, np.arange(1, 13)))
ds_woa_mon = xr.concat(ds_list, dim='time', compat='override', coords='minimal').squeeze().compute()
ds_woa_mon = ds_woa_mon[["s_an"]]
ds_woa_mon

In [ ]:
ds_woa_mon.s_an.isel(time=0).plot()

In [ ]:
woa_grid = regrid_tools.grid("latlon", nx=360, ny=180, lon0=-180.)
woa_grid

In [ ]:
era5_grid = regrid_tools.grid("latlon", nx=1440, ny=720, lon0=0.)
era5_grid

In [ ]:
regrid_woa_to_era5 = regrid_tools.regridder(woa_grid, era5_grid, method="conserve", clobber=False)
regrid_woa_to_era5

In [ ]:
ds_woa_mon_era5 = regrid_woa_to_era5(ds_woa_mon)
ds_woa_mon_era5

In [ ]:
salinity_data = np.concatenate([ds_woa_mon_era5.s_an.data for i in range(year0, yearf+1)], axis=0)
ds["sos"] = xr.DataArray(salinity_data, dims=ds.tos.dims, coords=ds.tos.coords)
ds

In [ ]:
ds = util.compute_fgn2(ds, scaleby).rename({'fgn2': 'SFN2_OCN'})
ds['SFN2_OCN'].attrs['units'] = 'mol/m^2/s'
ds

In [ ]:
region = xr.DataArray(['NET', 'SET'], dims=('region'), name='region')
masked_area = xr.concat([
    ds.area.where(ds.lat >= 20),
    ds.area.where(ds.lat <= -20),    
],
    dim=region,
)
masked_area.isel(region=0).plot()

In [ ]:
mols_to_Tmolmon = 86400. * 365. / 12 * 1e-12

vars_mean = ['tos', 'hfds']
vars_aint = ['SFN2_OCN']
with xr.set_options(keep_attrs=True):
    ds_mon_clim = ds.groupby('time.month').mean('time')    
    ds_mon_clim_region = (ds_mon_clim[vars_mean] * masked_area).sum(['lat', 'lon']) / masked_area.sum(['lat', 'lon'])
    ds_mon_clim_region = xr.merge([
        ds_mon_clim_region,
        (ds_mon_clim[vars_aint] * masked_area).sum(['lat', 'lon']),
    ])
    ds_mon_clim_region['SFN2_OCN'] = ds_mon_clim_region.SFN2_OCN * mols_to_Tmolmon
    ds_mon_clim_region.SFN2_OCN.attrs['units'] = 'Tmol mon${-1}$'
ds_mon_clim_region

In [ ]:
for v in ['hfds', 'tos', 'SFN2_OCN']:
    plt.figure()
    for i in range(ds_mon_clim_region.sizes['region']):
        ds_mon_clim_region[v].isel(region=i).plot(label=ds_mon_clim_region.region.isel(region=i).values)
    plt.legend();
    plt.title(v)

In [ ]:
(ds_mon_clim * masked_area).sum(['lat', 'lon']) / masked_area.sum(['lat', 'lon'])

In [ ]:
dst_grid = regrid_tools.grid("latlon", **config.config_dict["flux-dst-grid-kwargs"])
dst_grid

In [ ]:
regrid_obj = regrid_tools.regridder(era5_grid, dst_grid, method="conserve", clobber=False)
regrid_obj

In [ ]:
dso = util.generate_latlon_grid(**config.config_dict["flux-dst-grid-kwargs"])[["area"]]
dso['SFN2_OCN'] = regrid_obj(ds['SFN2_OCN'])
dso

In [ ]:
ds.SFN2_OCN.isel(time=0).plot()

In [ ]:
dso.SFN2_OCN.isel(time=0).plot()

In [ ]:
time, time_bnd = util.gen_midmonth_cftime_coord((year0, yearf), shift_time=shift_time)
dso['time'] = time
dso[time.bounds] = time_bnd
dso

In [ ]:
file_out = f"{config.flux_product_dir}/SFN2_OCN.era5-shf.shft_{shift_time:0.0f}d.scale_{scaleby:0.2f}.{year0}-{yearf}.nc"

dso.attrs = {}
dso.attrs['note'] = 'Heat flux from ERA5 Reanalysis: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means'
util.to_netcdf_clean(dso, file_out)

In [ ]:
curator = util.curate_flux_products()
curator.add_source(
        key=f"SFN2_OCN.era5_shf",
        urlpath=file_out,
        description='N2 flux computed from ERA5 heat flux',
    )